# WorkFlow

* Intro to Tensors
* Getting info from Tensors
* Manipulating Tensors
* Tensors and Numpy
* Using @tf.function (speed up regular python fucntion)
* Using GPU/TPU w/TensorFlow
* Excercise


### Intro to `Tensors`

In [1]:
# import tensorflow

import tensorflow as tf
print(tf.__version__)

2022-12-26 12:42:34.149355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0


In [2]:
# creating tensors with tf.contant()
scalar = tf.constant(7)
scalar
# has an empty shape
# tf.constant? ceates a constant tensor from a tensor-like object

2022-12-26 12:42:37.434363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# check for number of dimensions
scalar.ndim

0

In [4]:
# create a vector
vector = tf.constant([10,10]) # pass a python list to constant function
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# check dimension of vector
vector.ndim

1

In [6]:
# create a matrix
matrix = tf.constant([
    [10,7],
    [7,10]
])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
# check for dimension
matrix.ndim

2

In [8]:
another_matrix = tf.constant([[1.,4.],
[3.,5.],
[9., 12.]], dtype=tf.float16) # reduce data size occuiance on disk

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[ 1.,  4.],
       [ 3.,  5.],
       [ 9., 12.]], dtype=float16)>

In [9]:
another_matrix.ndim #number of elements in a shape

2

In [10]:
# create another tensor
tensor = tf.constant(
    [[[1,2,3,],
        [4,5,6]],
        [[7,8,9],
        [10,11,12]],
        [[13,14,15],
        [16,17,18]]]
)
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

### Creating Tensors with `tf.Variable`

In [12]:
changable_tensor = tf.Variable([10,7])
unchangable_tensor = tf.constant([10,7])

changable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [13]:
# lets chnag one of the elements in a chnagable tensor
# changable_tensor[0] = 13

In [14]:
# trying assign
changable_tensor[1].assign(15)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([10, 15], dtype=int32)>

### Creating Random Tensors

- The neural net always initializes itself with random weights/random tensors
- Adjusts the random weights wrt what the net learns

In [15]:
# create two random tensors
random_1 = tf.random.Generator.from_seed(42) # set the seed for reproducibility
random_1 = random_1.normal(shape=(4,2))
random_1

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ],
       [ 0.09988727, -0.50998646]], dtype=float32)>

In [16]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(4,2))
random_2

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ],
       [ 0.09988727, -0.50998646]], dtype=float32)>

In [17]:
# with both having a reproducibility value of 42
random_1, random_1, random_1 == random_2 # check hoe they compare

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [18]:
random_2 = tf.random.Generator.from_seed(2) # change the seed value to 2 and see how that compares
random_2 = random_2.normal(shape=(4,2))
random_2

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.1012345 , -0.2744976 ],
       [ 1.4204658 ,  1.2609464 ],
       [-0.43640924, -1.9633987 ],
       [-0.06452483, -1.056841  ]], dtype=float32)>

In [19]:
random_2, random_1, random_2 == random_1 # change the seed value and the reproducibility is different

(<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.1012345 , -0.2744976 ],
        [ 1.4204658 ,  1.2609464 ],
        [-0.43640924, -1.9633987 ],
        [-0.06452483, -1.056841  ]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ],
        [ 0.09988727, -0.50998646]], dtype=float32)>,
 <tf.Tensor: shape=(4, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False],
        [False, False]])>)

### Shuffling: 
* Inheret Order shouldn't affect learning

In [20]:
# not_shuffled = tf.random.Generator.from_seed(42)
# not_shuffled = not_shuffled.normal(shape=(3,2))
# not_shuffled

not_shuffled = tf.constant([[1,2],
[3,4],
[5,6]])
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [21]:
shuffled = tf.random.shuffle(not_shuffled)
shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[3, 4],
       [5, 6],
       [1, 2]], dtype=int32)>

In [22]:
not_shuffled, shuffled, not_shuffled == shuffled

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[3, 4],
        [5, 6],
        [1, 2]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False]])>)

In [23]:
# global set_seed
tf.random.set_seed(42) # global
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [24]:
# create a tensor array of all zeros
tf.zeros(shape=(2,4), dtype=tf.int32)

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int32)>

In [25]:
tf.ones([4,6], tf.float64)

<tf.Tensor: shape=(4, 6), dtype=float64, numpy=
array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])>

#### Convert a NumPy Array into a Tensor

In [26]:
# import numpy
import numpy as np
numpy_A = np.arange(1,25,dtype=np.int16)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int16)

In [27]:
# convert the Array into a Tensor
tensor_t = tf.constant(numpy_A)
tensor_t
tensor_tshaped = tf.constant(numpy_A, shape=(2,4,3))
tensor_tshaped

<tf.Tensor: shape=(2, 4, 3), dtype=int16, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18],
        [19, 20, 21],
        [22, 23, 24]]], dtype=int16)>

In [28]:
numpy_B = np.arange(12.3,31.1, dtype=np.float16)
numpy_B

array([12.3, 13.3, 14.3, 15.3, 16.3, 17.3, 18.3, 19.3, 20.3, 21.3, 22.3,
       23.3, 24.3, 25.3, 26.3, 27.3, 28.3, 29.3, 30.3], dtype=float16)

In [29]:
tensor_tb = tf.constant(numpy_B, dtype=tf.int16)
tensor_tb

<tf.Tensor: shape=(19,), dtype=int16, numpy=
array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30], dtype=int16)>

## Getting information from a tensor.
* Shape 
* Rank
* Axis/Dimension
* Size

In [42]:
# create a rank_4 tensor: rank - no. of tensor dimensions
rank_4_tensor = tf.zeros([2,3,4,5], dtype=tf.int32)
# see how it looks like
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=int32, numpy=
array([[[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]]], dtype=int32)>

In [45]:
# create a rank_2 tensor: no. of dimension
rank_2_tensor = tf.zeros([3,2], dtype=tf.int16)
rank_2_tensor

<tf.Tensor: shape=(3, 2), dtype=int16, numpy=
array([[0, 0],
       [0, 0],
       [0, 0]], dtype=int16)>

In [49]:
# get the 1st element of the tensor
rank_4_tensor[1]

<tf.Tensor: shape=(3, 4, 5), dtype=int32, numpy=
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]], dtype=int32)>

In [51]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [52]:
rank_4_tensor.ndim

4

In [53]:
tf.size(rank_4_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=120>

In [54]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]])

In [57]:
t.shape, t.ndim, tf.size(t)

(TensorShape([2, 2, 3]), 3, <tf.Tensor: shape=(), dtype=int32, numpy=12>)

In [58]:
t

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[1, 1, 1],
        [2, 2, 2]],

       [[3, 3, 3],
        [4, 4, 4]]], dtype=int32)>

In [60]:
# deep look into the attributes of our tensor
print("Datatype of Every Element: ", rank_4_tensor.dtype)
print("Number of Dimension (rank): ", rank_4_tensor.ndim)
print("Shape of Tensor: ",rank_4_tensor.shape)

Datatype of Every Element:  <dtype: 'int32'>
Number of Dimension (rank):  4
Shape of Tensor:  (2, 3, 4, 5)


In [62]:
print("Elements along the 0 axis: ",rank_4_tensor.shape[0])

Elements along the 0 axis:  2


In [63]:
print("Elements along the last axis: ", rank_4_tensor.shape[-1])

Elements along the last axis:  5


In [65]:
print("Total number of elements in our tensor: ",tf.size(rank_4_tensor).numpy())

Total number of elements in our tensor:  120
